In [ ]:
import pandas
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
import umap

In [ ]:
wdir = '/data1/users/jvogel/Allen_Human_Brain_Atlas/'
shtz = sorted(glob(os.path.join(wdir,'normalized_microarray_donor*/MExp_all_genes_ctr_for_donor')))
holder = []
for sht in shtz:
    holder.append(pandas.read_csv(sht,index_col=0))
bigdf = pandas.concat(holder,axis=1)
del(holder)
bigdf = bigdf.values

In [ ]:
rix = np.random.permutation(list(range(bigdf.shape[-1])))[:1000]
pca_mod = PCA(n_components=100).fit(bigdf[:,rix].T)
print('explained variances:', sum(pca_mod.explained_variance_ratio_))
tfm_mtx = pca_mod.transform(bigdf.T)

In [ ]:
gdf = pandas.read_csv(os.path.join(wdir,'gxp_correlation_wholebrain_results_NEW.csv'))
outdir = os.path.join(os.getcwd(),'umap_output')
nbrs = list(range(5,55,5))
dists = [0.001,0.01,0.1,0.5]
labs = [0 if gdf.loc[x,'slab_type'] == 'CX' else 1 if gdf.loc[x,'slab_type'] == 'BS' else 2 for x in gdf.index]
for nbr in nbrs:
    for dist in dists:
        print('working on nbr=%s, dist=%s'%(nbr,dist))
        embedding = umap.UMAP(n_components=3, n_neighbors=nbr, min_dist=dist).fit_transform(tfm_mtx)
        plt.close()
        fig = plt.figure()
        ax = Axes3D(fig)
        ax.scatter(embedding[:,0], embedding[:,1], embedding[:,2], c = np.array(labs), cmap='Spectral', s=5)
        plt.savefig(os.path.join(outdir,'umap_%s_%s.pdf'%(nbr,dist)))

In [ ]:
new_map = dict(zip(np.unique(labs),range(len(np.unique(labs)))))
newlabs = [new_map[x] for x in results.Label.values]
for nbr in nbrs:
    for dist in dists:
        print('working on nbr=%s, dist=%s'%(nbr,dist))
        embedding = umap.UMAP(n_components=3, n_neighbors=nbr, min_dist=dist).fit_transform(tfm_mtx)
        plt.close()
        fig = plt.figure()
        ax = Axes3D(fig)
        ax.scatter(embedding[:,0], embedding[:,1], embedding[:,2], c = newlabs, cmap='Spectral', s=5)
        plt.savefig(os.path.join(outdir,'dkt_umap_%s_%s.pdf'%(nbr,dist)))